### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Sport climbing at the Summer Olympics

Competition climbing made its Olympic debut at the 2020 Summer Olympics in Tokyo, Japan (postponed to 2021 due to the COVID-19 pandemic).

Two events were held, one each for men and women. The format controversially consisted of 1 single combined event with three disciplines: lead climbing, speed climbing and bouldering. The medals were determined based on best performance across all three disciplines

In [31]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_Staging'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [32]:
### url
Wiki_Climbing_Summer_Olympics_2020 = "https://en.wikipedia.org/wiki/Sport_climbing_at_the_2020_Summer_Olympics"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Climbing_Summer_Olympics_2020 = pd.read_html(Wiki_Climbing_Summer_Olympics_2020)
type(Dataset_Wiki_Climbing_Summer_Olympics_2020), len(Dataset_Wiki_Climbing_Summer_Olympics_2020)

(list, 9)

##### on récupère 1 list de 9 élts

In [4]:
type(Dataset_Wiki_Climbing_Summer_Olympics_2020[0])

pandas.core.frame.DataFrame

##### > Chaque élt est un DataFrame Pandas
##### ? Quel est le contenu chacun des 18 dataframes ? Lesquelles nous interessent ?

In [5]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[0]

,Competition climbing at the Games of the XXXII Olympiad,Competition climbing at the Games of the XXXII Olympiad.1
0,NaN,NaN
1,Venue,"Aomi Urban Sports Park, Tokyo"
2,Dates,3–6 August 2021
3,No. of events,2
4,Competitors,40 from 19 nations
5,2024 →,2024 →


######## [0] is dataset of summarized data about Sport Climbing at 2020 Summer Olympics dataset

In [30]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[1]

,Games,Events,Best nations
0,2020 Summer Olympics,2,Spain (1) Slovenia (1)
1,2024 Summer Olympics,4,Poland (1)


######## [1] is the Nb of Events at 2020 and 2024 Summer Olympics and Best Nations

In [6]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[2]

,Date,Aug 3,Aug 3.1,Aug 3.2,Aug 4,Aug 4.1,Aug 4.2,Aug 5,Aug 5.1,Aug 5.2,Aug 6,Aug 6.1,Aug 6.2
0,Men's,S Qualification: Speed climbing,B Qualification: Bouldering,L Qualification: Lead climbing,NaN,NaN,NaN,S Finals: Speed climbing,B Finals: Bouldering,L Finals: Lead climbing,NaN,NaN,NaN
1,Women's,NaN,NaN,NaN,S Qualification: Speed climbing,B Qualification: Bouldering,L Qualification: Lead climbing,NaN,NaN,NaN,S Finals: Speed climbing,B Finals: Bouldering,L Finals: Lead climbing
2,"S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead","S = Speed, B = Bouldering, L = Lead"


[2] is about Program of Climbing events at 2020 Summer Olympics

In [7]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[3]

,0,1,2,3
0,Q,Qualification,F,Finals


[3] > to Discard.

In [33]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[4]

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Slovenia,1,0,0,1
1,1,Spain,1,0,0,1
2,3,Japan*,0,1,1,2
3,4,United States,0,1,0,1
4,5,Austria,0,0,1,1
5,Totals (5 entries),Totals (5 entries),2,2,2,6


[4] is the Nations Medal table at 2020 Summer Olympics

In [34]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].columns

Index(['Rank', 'NOC', 'Gold', 'Silver', 'Bronze', 'Total'], dtype='object')

In [24]:
# Rename Column NOC to match with other competition tables
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].columns = ['Rank','Nation','Gold','Silver','Bronze','Total']
Dataset_Wiki_Climbing_Summer_Olympics_2020[4]                                                                                                           

,Rank,Nation,Gold,Silver,Bronze,Total
0,1,Slovenia,1,0,0,1
1,1,Spain,1,0,0,1
2,3,Japan,0,1,1,2
3,4,United States,0,1,0,1
4,5,Austria,0,0,1,1
5,Totals (5 entries),Totals (5 entries),2,2,2,6


In [35]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].iloc[2,1]

'Japan*'

In [36]:
# Remove * on Japan
# # Translate Country Names in French to Match with other Tables de Data (géographiques, Démographiques, ...)
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].iloc[0,1] = 'Slovénie'
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].iloc[1,1] = 'Espagne'
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].iloc[2,1] = 'Japan'
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].iloc[3,1] = 'États-Unis'
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].iloc[3,1] = 'Autriche'


In [37]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[4]

,Rank,NOC,Gold,Silver,Bronze,Total
0,1,Slovénie,1,0,0,1
1,1,Espagne,1,0,0,1
2,3,Japan,0,1,1,2
3,4,Autriche,0,1,0,1
4,5,Austria,0,0,1,1
5,Totals (5 entries),Totals (5 entries),2,2,2,6


In [38]:
Dataset_Wiki_Climbing_Summer_Olympics_2020[4].dtypes

Rank      object
NOC       object
Gold       int64
Silver     int64
Bronze     int64
Total      int64
dtype: object

In [39]:
Wiki_IFSC_Climbing_Summer_Olympics_2020_Countries_Total_Ranking_Df4 = Dataset_Wiki_Climbing_Summer_Olympics_2020[4]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_IFSC_Climbing_Summer_Olympics_2020_Countries_Total_Ranking_Df4
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rank VARCHAR(50),
                                        Nation VARCHAR(50),
                                        Gold INT,
                                        Silver INT,
                                        Bronze INT,
                                        Total INT                                                                  
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_IFSC_Climbing_Summer_Olympics_2020_Countries_Total_Ranking_Df4.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_IFSC_Climbing_Summer_Olympics_2020_Countries_Total_Ranking_Df4',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

err: ('42S01', "[42S01] [Microsoft][ODBC SQL Server Driver][SQL Server]There is already an object named 'Wiki_IFSC_Climbing_Summer_Olympics_2020_Countries_Total_Ranking_Df4' in the database. (2714) (SQLExecDirectW)")


In [10]:
Dataset_Wiki_Climbing_2020_Summer_Olympics[5]

,Event,Gold,Silver,Bronze
0,Men's combined details,Alberto Ginés López Spain,Nathaniel Coleman United States,Jakob Schubert Austria
1,Women's combined details,Janja Garnbret Slovenia,Miho Nonaka Japan,Akiyo Noguchi Japan


[5] > to Discard, already available

In [11]:
Dataset_Wiki_Climbing_2020_Summer_Olympics[6]

,Event,Round,Climber,Nation,Time,Date,Record
0,Men's combined (speed),Qualification,Bassa Mawem,France,5.45,3 August,OR[17]
1,Women's combined (speed),Qualification,Aleksandra Mirosław,Poland,6.97,4 August,OR[18]
2,Women's combined (speed),Final,Aleksandra Mirosław,Poland,6.84,6 August,WR[19]


[6] > to Discard.

In [12]:
Dataset_Wiki_Climbing_2020_Summer_Olympics[7]

,"vteEvents at the 2020 Summer Olympics (Tokyo, Japan)","vteEvents at the 2020 Summer Olympics (Tokyo, Japan).1"
0,Archery Artistic swimming Athletics Badminton ...,Archery Artistic swimming Athletics Badminton ...
1,Chronological summary Medal table List of meda...,Chronological summary Medal table List of meda...
